
# AWS Lab Exercise — IAM Role for EC2 Accessing S3 (Console Based)

### 🎯 Objective
Learn how to use **IAM Roles** to grant permissions to AWS services.  
In this lab, you will create a role that allows an **EC2 instance** to access **S3** — without using any access keys.

---

### 🧭 Overview
You will perform the following steps:

1. Create an IAM Role for EC2 with S3 ReadOnly permissions.  
2. Launch an EC2 instance with that role.  
3. Connect to EC2 and verify access to S3 using AWS CLI.  
4. Validate that upload/delete actions are restricted.  

---



## Step 1: Create an IAM Role for EC2

1. Navigate to **IAM → Roles → Create role**.  
2. Under **Trusted entity type**, select **AWS Service**.  
3. Under Use Case, Choose **EC2** → click **Next**.  
4. Under **Permissions policies**, attach:  
   - `AmazonS3ReadOnlyAccess`  
5. Click **Next**, then name the role:  
   - `EC2_S3ReadOnlyRole`  
6. Review and click **Create role**.

✅ **Concept:** The EC2_S3ReadOnlyRole grants temporary credentials to any EC2 instance it is attached to, allowing it to list and read data from S3.



## Step 2: Launch an EC2 Instance with the IAM Role

1. Go to **EC2 → Launch Instance**.  
2. Choose **Amazon Linux 2 AMI**.  
3. Select instance type **t2.micro**.  
4. Under **Advanced Details → IAM Instance Profile**, choose:  
   - `EC2_S3ReadOnlyRole`  
5. Select your existing **key pair** (e.g., `my-key`).  
6. Keep the region as **us-east-1**.  
7. Launch the instance.

✅ **Note:** Attaching the role here ensures the EC2 instance automatically receives permissions to access S3 without configuring access keys.



## Step 3: Connect to EC2 Instance

Once the instance is running, connect using SSH:

```bash
ssh -i my-key.pem ec2-user@<public-dns>
```

### Verify IAM Identity
```bash
aws sts get-caller-identity
```
Expected output should display the **role ARN**, such as:
```
arn:aws:iam::<account-id>:role/EC2_S3ReadOnlyRole
```



## Step 4: Access S3 Data from EC2

Run the following commands to verify access to S3:

### List S3 Buckets
```bash
aws s3 ls
```

You should see the list of available buckets.

### Download a File from S3
```bash
aws s3 cp s3://company-data-lab-yourname/orders.csv /tmp/
```
✅ File should download successfully.

### View the File
```bash
cat /tmp/orders.csv
```

✅ The file contents should display properly.



## Step 5: Validate Access Restrictions

Try performing an action not allowed by the role:

### Attempt to Delete a File
```bash
aws s3 rm s3://company-data-lab-yourname/orders.csv
```
Expected output:
```
An error occurred (AccessDenied) when calling the DeleteObject operation:
Access Denied
```

✅ **Observation:** The `AmazonS3ReadOnlyAccess` policy only permits viewing and downloading objects, not deletion or upload.



---

## Reflection

| AWS Entity | Type | Credential | Access to S3 | Persistence |
|-------------|-------|-------------|--------------|-------------|
| IAM User | Permanent | Access Key | Depends on policy | Long-term |
| IAM Role | Temporary | No key needed | Based on attached policy | Temporary |

✅ **Key Learnings:**
- IAM Roles provide **temporary, secure access** without storing credentials.  
- Roles can be attached to AWS services like EC2, Lambda, Glue, or Redshift.  
- Managed policies like `AmazonS3ReadOnlyAccess` control exactly what actions can be performed.  
- EC2 can interact with S3 safely through the **Instance Profile** mechanism.

---

### 🧠 **Summary**
- You created an IAM Role for EC2.  
- Attached it during EC2 launch.  
- Verified access to S3 without credentials.  
- Observed restricted behavior when trying to modify or delete data.

---
